In [ ]:
import pandas as pd
import pdfplumber
from fuzzywuzzy import fuzz

import os
os.chdir("../")

X_VALUE_CO_FOUNDERS = 257.44449036
X_VALUE_WHATS_DIFFERENT = 245.53287961199996
X_VALUE_EMPLOYEES = 266.28213704399997
X_VALUE_COVER = 29.971149623999995
X_VALUE_RESPONSES = 119.88459849599998

PDF_DIR = "./data/Startups/Factsheets"
JSON = "./data/texts.json"

RESPONSE_1 = "1. Your company/vision/mission in a tweet ! *"
RESPONSE_2 = "2. Which sector/subsector of Fintech do you operate in?"
RESPONSE_3 = "3. SStage of company *"
RESPONSE_4 = "4. Which problem does your company solve? *"
RESPONSE_5 = "5. What sparked the founder(s) to set up the company? *"
RESPONSE_6 = "6. Please describe the business model of your company, including the main sources of revenue. *"
RESPONSE_7 = "7. What is your USP versus traditional competitors or those in the digital world? *"
RESPONSE_8 = "8. Which target group(s) in which markets do you address primarily? *"
RESPONSE_9 = "9. Please share any other office locations, you might have!"
RESPONSE_10 = "10. Are you a German startup?"
RESPONSE_11 = "11. If you are a foreign startup: Do you have clients in Germany? How many? In B2C/B2B?"
RESPONSE_12 = "12. Please provide a brief competition landscape for your core markets. *"
RESPONSE_13 = "13. Did you pivot in the past and why?"
RESPONSE_14 = "14. Which business area in the last year consumed most of your management attention? *"
RESPONSE_15 = "15. What protects your business from new market entrants or copycats? *"
RESPONSE_16 = "16. After your next financing round, will the founders jointly hold above or below 50% of the"
RESPONSE_17 = "17. Anything else you would want to add or let us know?"
RESPONSE_18 = "18. How do you integrate ESG standards in your business model? *"
RESPONSE_19 = "19. When scaling the company, how do you ensure that the corporate culture does not suffer and"
RESPONSE_20 = "20. Please upload your startup logo. *"
RESPONSE_21 = "21. Did your firm apply to FTGA before? If yes, which year? *"

In [ ]:
def get_texts(pdf):
    pdf_path = os.path.join(PDF_DIR, pdf)

    with pdfplumber.open(pdf_path) as pdf:
        lines = []
        words = []
        for page in pdf.pages:
            lines += page.extract_text().split("\n")
            words += page.extract_words()
        
        pdf.close()
    
    description_lines, difference_lines, response_1_lines, response_2_lines, response_3_lines, response_4_lines, response_5_lines, response_6_lines, response_7_lines, response_8_lines, response_9_lines, response_10_lines, response_11_lines, response_12_lines, response_13_lines, response_14_lines, response_15_lines, response_16_lines, response_17_lines, response_18_lines, response_19_lines, response_20_lines, response_21_lines = get_lines(lines)
    description_words, difference_words, response_1_words, response_2_words, response_3_words, response_4_words, response_5_words, response_6_words, response_7_words, response_8_words, response_9_words, response_10_words, response_11_words, response_12_words, response_13_words, response_14_words, response_15_words, response_16_words, response_17_words, response_18_words, response_19_words, response_20_words, response_21_words = get_words(words)

    description = get_description(description_lines, description_words)
    difference = get_difference(difference_lines, difference_words)

    response1 = get_responses(response_1_lines, response_1_words)
    response2 = get_responses(response_2_lines, response_2_words)
    response3 = get_responses(response_3_lines, response_3_words)
    response4 = get_responses(response_4_lines, response_4_words)
    response5 = get_responses(response_5_lines, response_5_words)
    response6 = get_responses(response_6_lines, response_6_words)
    response7 = get_responses(response_7_lines, response_7_words)
    response8 = get_responses(response_8_lines, response_8_words)
    response9 = get_responses(response_9_lines, response_9_words)
    response10 = get_responses(response_10_lines, response_10_words)
    response11 = get_responses(response_11_lines, response_11_words)
    response12 = get_responses(response_12_lines, response_12_words)
    response13 = get_responses(response_13_lines, response_13_words)
    response14 = get_responses(response_14_lines, response_14_words)
    response15 = get_responses(response_15_lines, response_15_words)
    response16 = get_responses(response_16_lines, response_16_words)
    response17 = get_responses(response_17_lines, response_17_words)
    response18 = get_responses(response_18_lines, response_18_words)
    response19 = get_responses(response_19_lines, response_19_words)
    response20 = get_responses(response_20_lines, response_20_words)
    response21 = get_responses(response_21_lines, response_21_words)

    return description, difference, response1, response2, response3, response4, response5, response6, response7, response8, response9, response10, response11, response12, response13, response14, response15, response16, response17, response18, response19, response20, response21

def get_lines(lines):
    return filter_word_or_lines(lines, get_lines_equal)

def get_lines_equal(line, text):
    similarity_ratio = fuzz.ratio(line, text)
    return similarity_ratio >= 90

def get_words(words):
    description, difference, response_1, response_2, response_3, response_4, response_5, response_6, response_7, response_8, response_9, response_10, response_11, response_12, response_13, response_14, response_15, response_16, response_17, response_18, response_19, response_20, response_21 = filter_word_or_lines(words, get_words_equal)
    return filter(description), filter(difference), filter(response_1, "RESPONSES"), filter(response_2, "RESPONSES"),  filter(response_3, "RESPONSES"),  filter(response_4, "RESPONSES"),  filter(response_5, "RESPONSES"),  filter(response_6, "RESPONSES"),  filter(response_7, "RESPONSES"),  filter(response_8, "RESPONSES"),  filter(response_9, "RESPONSES"),  filter(response_10, "RESPONSES"),  filter(response_11, "RESPONSES"),  filter(response_12, "RESPONSES"),  filter(response_13, "RESPONSES"),  filter(response_14, "RESPONSES"),  filter(response_15, "RESPONSES"), filter(response_16, "RESPONSES"), filter(response_17, "RESPONSES"), filter(response_18, "RESPONSES"), filter(response_19, "RESPONSES"), filter(response_20, "RESPONSES"), filter(response_21, "RESPONSES")

def get_words_equal(word, text):
    if text == "CO-FOUNDERS":
        return word["x0"] == X_VALUE_CO_FOUNDERS and word["text"] == text
    if text == "WHAT'S DIFFERENT":
        return word["x0"] == X_VALUE_WHATS_DIFFERENT and word["text"] == "WHAT'S"
    if text == "EMPLOYEES":
        return word["x0"] == X_VALUE_EMPLOYEES and word["text"] == text
    if text == "RESPONSES":
        return word["x0"] == X_VALUE_RESPONSES and word["text"] == text
    
    text = text.split(". ")[0]
    return word["x0"] == X_VALUE_RESPONSES and word["text"].startswith(text)
    
def filter(words, where="COVER"):
    if where == "COVER":
        X_VALUE = X_VALUE_COVER
    elif where == "RESPONSES":
        X_VALUE = X_VALUE_RESPONSES
    return [word for word in words if word["x0"] == X_VALUE]

def filter_word_or_lines(lines_words, equal_function):
    description = []
    difference = []
    response_1 = []
    response_2 = []
    response_3 = []
    response_4 = []
    response_5 = []
    response_6 = []
    response_7 = []
    response_8 = []
    response_9 = []
    response_10 = []
    response_11 = []
    response_12 = []
    response_13 = []
    response_14 = []
    response_15 = []
    response_16 = []
    response_17 = []
    response_18 = []
    response_19 = []
    response_20 = []
    response_21 = []

    in_description = True
    in_difference = False
    in_response_1 = False
    in_response_2 = False
    in_response_3 = False
    in_response_4 = False
    in_response_5 = False
    in_response_6 = False
    in_response_7 = False
    in_response_8 = False
    in_response_9 = False
    in_response_10 = False
    in_response_11 = False
    in_response_12 = False
    in_response_13 = False
    in_response_14 = False
    in_response_15 = False
    in_response_16 = False
    in_response_17 = False
    in_response_18 = False
    in_response_19 = False
    in_response_20 = False
    in_response_21 = False
    for line_word in lines_words:
        if equal_function(line_word, "CO-FOUNDERS"):
            in_description = False
            continue
        if equal_function(line_word, "WHAT'S DIFFERENT"):
            in_difference = True
            continue
        if equal_function(line_word, "EMPLOYEES") or equal_function(line_word, "RESPONSES"):
            in_difference = False
            continue
        if equal_function(line_word, RESPONSE_1):
            in_response_1 = True
            continue
        if equal_function(line_word, RESPONSE_2):
            in_response_1 = False
            in_response_2 = True
            continue
        if equal_function(line_word, RESPONSE_3):
            in_response_2 = False
            in_response_3 = True
            continue
        if equal_function(line_word, RESPONSE_4):
            in_response_3 = False
            in_response_4 = True
            continue
        if equal_function(line_word, RESPONSE_5):
            in_response_4 = False
            in_response_5 = True
            continue
        if equal_function(line_word, RESPONSE_6):
            in_response_5 = False
            in_response_6 = True
            continue
        if equal_function(line_word, RESPONSE_7):
            in_response_6 = False
            in_response_7 = True
            continue
        if equal_function(line_word, RESPONSE_8):
            in_response_7 = False
            in_response_8 = True
            continue
        if equal_function(line_word, RESPONSE_9):
            in_response_8 = False
            in_response_9 = True
            continue
        if equal_function(line_word, RESPONSE_10):
            in_response_9 = False
            in_response_10 = True
            continue
        if equal_function(line_word, RESPONSE_11):
            in_response_10 = False
            in_response_11 = True
            continue
        if equal_function(line_word, RESPONSE_12):
            in_response_11 = False
            in_response_12 = True
            continue
        if equal_function(line_word, RESPONSE_13):
            in_response_12 = False
            in_response_13 = True
            continue
        if equal_function(line_word, RESPONSE_14):
            in_response_13 = False
            in_response_14 = True
            continue
        if equal_function(line_word, RESPONSE_15):
            in_response_14 = False
            in_response_15 = True
            continue
        if equal_function(line_word, RESPONSE_16):
            in_response_15 = False
            in_response_16 = True
            continue
        if equal_function(line_word, RESPONSE_17):
            in_response_16 = False
            in_response_17 = True
            continue
        if equal_function(line_word, RESPONSE_18):
            in_response_17 = False
            in_response_18 = True
            continue
        if equal_function(line_word, RESPONSE_19):
            in_response_18 = False
            in_response_19 = True
            continue
        if equal_function(line_word, RESPONSE_20):
            in_response_19 = False
            in_response_20 = True
            continue
        if equal_function(line_word, RESPONSE_21):
            in_response_20 = False
            in_response_21 = True
            continue
        if in_description:
            description.insert(0, line_word)
        if in_difference:
            difference.append(line_word)
        if in_response_1:
            response_1.append(line_word)
        if in_response_2:
            response_2.append(line_word)
        if in_response_3:
            response_3.append(line_word)
        if in_response_4:
            response_4.append(line_word)
        if in_response_5:
            response_5.append(line_word)
        if in_response_6:
            response_6.append(line_word)
        if in_response_7:
            response_7.append(line_word)
        if in_response_8:
            response_8.append(line_word)
        if in_response_9:
            response_9.append(line_word)
        if in_response_10:
            response_10.append(line_word)
        if in_response_11:
            response_11.append(line_word)
        if in_response_12:
            response_12.append(line_word)
        if in_response_13:
            response_13.append(line_word)
        if in_response_14:
            response_14.append(line_word)
        if in_response_15:
            response_15.append(line_word)
        if in_response_16:
            response_16.append(line_word)
        if in_response_17:
            response_17.append(line_word)
        if in_response_18:
            response_18.append(line_word)
        if in_response_19:
            response_19.append(line_word)
        if in_response_20:
            response_20.append(line_word)
        if in_response_21:
            response_21.append(line_word)

    return description, difference, response_1, response_2, response_3, response_4, response_5, response_6, response_7, response_8, response_9, response_10, response_11, response_12, response_13, response_14, response_15, response_16, response_17, response_18, response_19, response_20, response_21

def get_description(lines, words):
    description = ""
    if len(words) > 0:
        for word in words:
            while not lines[0].startswith(word["text"]):
                lines = lines[1:]
            description = lines[0] + " " + description
    return description

def get_difference(lines, words):
    difference = ""
    if len(words) > 0:
        for word in words:
            while not lines[0].startswith(word["text"]):
                lines = lines[1:]
            difference += " " + lines[0]
    return difference

def get_responses(lines, words):
    return "\n".join(lines)

In [ ]:
pdf_files = pd.DataFrame([file for file in os.listdir(PDF_DIR) if file.lower().endswith(".pdf")], columns=["Filename"])
pdf_files.head()

In [ ]:
pdf_files[['Description', 'Difference', RESPONSE_1, RESPONSE_2, RESPONSE_3, RESPONSE_4, RESPONSE_5, RESPONSE_6, RESPONSE_7, RESPONSE_8, RESPONSE_9, RESPONSE_10, RESPONSE_11, RESPONSE_12, RESPONSE_13, RESPONSE_14, RESPONSE_15, RESPONSE_16, RESPONSE_17, RESPONSE_18, RESPONSE_19, RESPONSE_20, RESPONSE_21]] = pdf_files['Filename'].apply(get_texts).apply(pd.Series)
pdf_files.to_json(JSON)